In [1]:
start_date = '2010-01-01'
end_date = '2021-08-31'

s3_bucket = 'YOUR_BUCKET'
parquet_s3_location = "data/repo/esg/daily_dates_with_weekends/daily_dates_with_weekends.parquet"

# ===================================
# DON'T CHANGE ANYTHING BELOW HERE
# ===================================

In [2]:
import boto3
from dateutil.rrule import DAILY, rrule, MO, TU, WE, TH, FR, SA, SU
from datetime import datetime
from io import BytesIO
import pandas as pd

In [3]:
# Don't let panda hide any columns
pd.options.display.max_columns = 0

### Generate Daily Time Series

In [4]:
# Convert date strings to datetime types
start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
end_datetime = datetime.strptime(end_date, '%Y-%m-%d')

In [5]:
# Create a rule for all weekdays
weekdays = rrule(DAILY, dtstart=start_datetime, until=end_datetime, byweekday=(MO,TU,WE,TH,FR,SA,SU))

In [6]:
# Generate pandas dataframe of all weekdays
dates_df = pd.DataFrame(list(weekdays), columns=['as_of_date'])

In [7]:
dates_df.head()

,as_of_date
0,2010-01-01
1,2010-01-02
2,2010-01-03
3,2010-01-04
4,2010-01-05


### Write Daily Dates to S3

In [8]:
s3_resource = boto3.resource('s3')  

In [9]:
parquet_buffer = BytesIO()
dates_df.to_parquet(parquet_buffer, index=False)
s3_resource.Object(s3_bucket, parquet_s3_location).put(Body=parquet_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'XN2V4NY2JGP221TQ',
  'HostId': 'VYlyqjZMKl0vaEbQ7+6OcrgUOjsIW3YLaOfSiA1kyV2dI8dOEd84SfP6VMtvu53YN1cjd6OayNM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'VYlyqjZMKl0vaEbQ7+6OcrgUOjsIW3YLaOfSiA1kyV2dI8dOEd84SfP6VMtvu53YN1cjd6OayNM=',
   'x-amz-request-id': 'XN2V4NY2JGP221TQ',
   'date': 'Tue, 31 Aug 2021 20:46:04 GMT',
   'x-amz-version-id': 'DRuXOWv2ZJLzz6LPD3JN5Nt1tB6zT5r2',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"2caffbdc91b93f587d2b0a9ed3f6592e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2caffbdc91b93f587d2b0a9ed3f6592e"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'DRuXOWv2ZJLzz6LPD3JN5Nt1tB6zT5r2'}